In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
from pathlib import Path
import random

import ultralytics as ul

import bb
import tt

LOG = logging.getLogger(__name__)
tt.logging_init()

In [15]:
data_path = Path.home() / "src/data"
mc_data_path = data_path / "minecraft/mobs/info.json"
dset = bb.Dataset.load(mc_data_path)
df = dset.to_df()

In [16]:
dset.dataset_stats()

num_images                    184
num_bboxes                    298
avg_bboxes_per_image     1.619565
num_categories                  9
most_common_category       zombie
least_common_category    enderman
avg_bbox_width           0.154801
avg_bbox_height          0.197132
avg_bbox_area            0.043763
min_bbox_area            0.000581
max_bbox_area            0.566857
dtype: object

In [17]:
dset.category_stats()

,count,avg_width,avg_height,avg_area
category,,,,
zombie,55,0.1409,0.2418,0.0553
pig,44,0.1555,0.1377,0.0298
cow,42,0.1643,0.1900,0.0483
chicken,36,0.1305,0.1546,0.0270
sheep,34,0.1827,0.2044,0.0542
skeleton,32,0.1267,0.2457,0.0482
spider,25,0.2417,0.1650,0.0510
creeper,18,0.1201,0.2384,0.0344
enderman,12,0.1224,0.2175,0.0336


# Yolo

In [ ]:
data_path = Path.home() / "src/data"
mc_data_path = data_path / "minecraft/mobs/info.json"
yolo_path = data_path / "yolo"
models_path = yolo_path / "models"

ul.settings.update(
    {
        "datasets_dir": str(yolo_path / "datasets"),
        "weights_dir": str(yolo_path / "weights"),
        "runs_dir": str(yolo_path / "runs"),
    }
)

random.seed(325)

In [ ]:
dset = bb.Dataset.load(mc_data_path)
dset.to_yolo(yolo_path, 80)

In [ ]:
model = ul.YOLO(models_path / "yolo11n.pt")

In [ ]:

train_results = model.train(data=yolo_path / "data.yaml", epochs=100, imgsz=640, device="mps")


In [ ]:
dset.images[0].full_path

In [ ]:
images = [ir.full_path for ir in dset.images]
results = model(images, device="mps")

In [ ]:
results[10].show()